In [1]:
!pip install transformers accelerate peft datasets torchvision bitsandbytes

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from peft import get_peft_model, LoraConfig, TaskType
from PIL import Image
from io import BytesIO
import requests
import torch.nn.functional as F
from tqdm import tqdm

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv("product_data.csv")

# Fill missing values
df["product_title"] = df["product_title"].fillna("")
df["product_description"] = df["product_description"].fillna("")

# Define keywords for filtering product titles and descriptions
keywords = ['nail', 'shampoo', 'conditioner', 'eye', 'lip', 'ear', 'nose', 'beauty', 'cosmetic', 'hair', 'skin','hand', 'leg', 'oil' ,'makeup', 'lotion', 'cream', 'cleanser', 'moisturizer']

# Filter rows where product_title or product_description contains any of the keywords
def contains_keywords(text, keywords):
    return any(keyword.lower() in text.lower() for keyword in keywords)

# Apply the filter
df_filtered = df[df["product_title"].apply(lambda x: contains_keywords(x, keywords)) | 
                 df["product_description"].apply(lambda x: contains_keywords(x, keywords))]

# Clean and format product text
def create_product_text(row):
    title = row["product_title"].strip()
    description = row["product_description"].strip()
    
    # If there's a description, include it; otherwise, just include the title
    if description:
        full_text = f"Product title is: {title}\nProduct description is: {description}"
    else:
        full_text = f"Product title is: {title}"
    
    return full_text[:512]  # limit text to 512 characters (adjust length if needed)

df_filtered["product_text"] = df_filtered.apply(create_product_text, axis=1)

# Keep only rows where product_text and product_image_url are not empty
df_cleaned = df_filtered[
    df_filtered["product_text"].str.strip().astype(bool) & 
    df_filtered["product_image_url"].str.strip().astype(bool)
].reset_index(drop=True)


/tmp/ipykernel_1225847/1365325199.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["product_text"] = df_filtered.apply(create_product_text, axis=1)


In [4]:

df_train = df_cleaned.sample(20000)

In [5]:
class ProductCLIPDataset(Dataset):
    def __init__(self, df):
        self.texts = df["product_text"].tolist()
        self.urls  = df["product_image_url"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        url  = self.urls[idx]
        try:
            img = Image.open(BytesIO(requests.get(url, timeout=5).content)).convert("RGB")
        except:
            img = Image.new("RGB",(224,224),"white")
        return {"text": text, "image": img}

In [6]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
def collate_fn(batch):
    texts  = [ex["text"]  for ex in batch]
    images = [ex["image"] for ex in batch]
    enc = processor(text=texts,
                    images=images,
                    return_tensors="pt",
                    padding=True,
                    truncation=True)
    return enc

dataset = ProductCLIPDataset(df_train)
loader  = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
lora_cfg = LoraConfig(
    r=8, lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION
)
model = get_peft_model(base, lora_cfg).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
import time
start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Move all to device
        batch = {k:v.to(device) for k,v in batch.items()}

        # 1) Get embeddings
        text_embs  = model.get_text_features(**{k:batch[k] for k in ["input_ids","attention_mask"]})
        image_embs = model.get_image_features(pixel_values=batch["pixel_values"])

        # 2) Normalize
        text_embs  = F.normalize(text_embs,  p=2, dim=-1)
        image_embs = F.normalize(image_embs, p=2, dim=-1)

        # 3) Similarity logits
        logits_per_text  = text_embs @ image_embs.t()
        logits_per_image = logits_per_text.t()

        # 4) Contrastive loss
        B = logits_per_text.size(0)
        labels = torch.arange(B, device=device)
        loss_t2i = F.cross_entropy(logits_per_text, labels)
        loss_i2t = F.cross_entropy(logits_per_image, labels)
        loss = (loss_t2i + loss_i2t) / 2

        # 5) Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg = total_loss / len(loader)
    print(f"Epoch {epoch+1} avg loss: {avg:.4f}")
print(time.time() -  start_time)

Epoch 1/10: 100%|███████████████████████████| 157/157 [1:10:18<00:00, 26.87s/it]


Epoch 1 avg loss: 4.6583


Epoch 2/10: 100%|███████████████████████████| 157/157 [1:07:04<00:00, 25.63s/it]


Epoch 2 avg loss: 4.4892


Epoch 3/10: 100%|███████████████████████████| 157/157 [1:03:47<00:00, 24.38s/it]


...

Epoch 5/10: 100%|███████████████████████████| 157/157 [1:05:50<00:00, 25.16s/it]


Epoch 5 avg loss: 4.2533


Epoch 6/10: 100%|███████████████████████████| 157/157 [1:04:20<00:00, 24.59s/it]


Epoch 6 avg loss: 4.2296


Epoch 7/10:  44%|█████████████▏                | 69/157 [28:33<35:11, 24.00s/it]

In [ ]:
model.save_pretrained("clip-lora-beauty-20k")
processor.save_pretrained("clip-lora-beauty-20k")

In [ ]:
asdf

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from peft import PeftModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load base CLIP
base_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# Load your fine-tuned adapter
model = PeftModel.from_pretrained(base_model, "clip-lora-beauty-full")
model = model.to(device).eval()

# Load processor
processor = CLIPProcessor.from_pretrained("clip-lora-beauty-full")


In [ ]:
from tqdm import tqdm
import numpy as np

text_embeddings = []
image_embeddings = []

for i, row in tqdm(df_meta_clean.iterrows(), total=len(df_meta_clean)):
    text = row['product_text']
    url  = row['product_image_url']

    # Image loading
    try:
        image = Image.open(BytesIO(requests.get(url, timeout=5).content)).convert("RGB")
    except:
        image = Image.new("RGB", (224, 224), "white")

    inputs = processor(text=text, images=image, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        t_emb = model.get_text_features(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        i_emb = model.get_image_features(pixel_values=inputs["pixel_values"])

    text_embeddings.append(t_emb.cpu().numpy())
    image_embeddings.append(i_emb.cpu().numpy())

text_embeddings = np.vstack(text_embeddings)
image_embeddings = np.vstack(image_embeddings)


In [ ]:
import numpy as np
np.save("product_text_embeddings_finetuned_full.npy", np.stack(text_embeddings))
np.save("product_image_embeddings_finetuned_full.npy", np.stack(image_embeddings))

In [ ]:
def recommend(query_text=None, query_image_url=None, alpha=0.5, top_k=5):
    assert query_text or query_image_url, "Need at least text or image"

    t_emb, i_emb = None, None
    
    if query_text:
        t_in = processor(text=query_text, return_tensors="pt", truncation=True).to(device)
        with torch.no_grad():
            t_emb = model.get_text_features(**t_in).cpu()  # [1, D]

    if query_image_url:
        img = Image.open(BytesIO(requests.get(query_image_url, timeout=10).content)).convert("RGB")
        i_in = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            i_emb = model.get_image_features(**i_in).cpu()# [1, D]

    sims = None

    if t_emb is not None and i_emb is not None:
        sim_text = cosine_similarity(t_emb, text_embeddings)[0]
        sim_image = cosine_similarity(i_emb, image_embeddings)[0]
        sims = alpha * sim_image + (1 - alpha) * sim_text

    elif t_emb is not None:
        sim_text = cosine_similarity(t_emb, text_embeddings)[0]
        sim_image = cosine_similarity(t_emb, image_embeddings)[0]
        sims = alpha * sim_image + (1 - alpha) * sim_text

    elif i_emb is not None:
        sim_text = cosine_similarity(i_emb, text_embeddings)[0]
        sim_image = cosine_similarity(i_emb, image_embeddings)[0]
        sims = alpha * sim_text + (1 - alpha) * sim_image

    idxs = sims.argsort()[::-1][:top_k]
    results = df_meta_clean.iloc[idxs][['product_title', 'product_image_url']].copy()
    results['similarity_score'] = sims[idxs]
    return results.reset_index(drop=True)

In [ ]:
import numpy as np
recommend(query_text="photo finish Professional airbrush makeup")

In [ ]:
query_image_url = "https://temptupro.com/cdn/shop/products/s-one-essential-airbrush-kit-hero_2.jpg?v=1743181132&width=1780"

In [ ]:
recommend(query_image_url=query_image_url)

In [ ]:
recommend(query_text=" temptu airbrush makeup kit with compressor", query_image_url= query_image_url)

In [ ]:
list(df_meta_clean['product_title'].sample(50))